In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv("Electoral_College.csv")
df = df.sort_values(by="Electoral_College_Votes",ascending=False)
df.head()

,Full_State,Abb_State,Electoral_College_Votes
4,California,CA,54
43,Texas,TX,40
9,Florida,FL,30
32,New York,NY,28
38,Pennsylvania,PA,19


In [3]:
electoral_college = [270] + df["Electoral_College_Votes"].to_list()

In [4]:
def BPI_raw(game,i):
    '''
    A dynamic programming algorithm for calculating the raw Banzhaf power index of player i in a weighted voter game.
    
    input:   list [q,w_1,w_2,...,w_n] representing a weighted voter game with weights in descending order;
                 we assume that w_i < q for all i
             int i, the index of a player
             
    output:  int b, the raw Banzhaf power index of player i
    '''
    q = game[0]
    n = len(game) - 1
    w_i = game[i]
    weights = game[1:]
    coalitions = [[w for i in range(np.sum([np.array(weights) == w]))] for w in sorted(set(weights),reverse=True)]
    z = len(coalitions)
    dp_array = [[0 for i in range(n)] for i in range(q)]
    dp_array[0][0] = 1
    ts = 0
    b = 0
    alpha = [0 for i in range(n)]
    
    for x in range(z):
        y_0 = len(coalitions[x])
        if w_i == coalitions[x][0]:
            y_0 = y_0 - 1
        for t in range(ts+1)[::-1]:
            for w in range(alpha[t]+1)[::-1]:
                if dp_array[w][t] > 0:
                    c_0 = 1
                    for y in range(1,y_0+1):
                        c_0 = c_0 * float(y_0-y+1) / y
                        c_add = round(dp_array[w][t] * c_0)
                        w_temp = w + coalitions[x][0] * y
                        if w_temp <= q-1:
                            dp_array[w_temp][t+y] = dp_array[w_temp][t+y] + c_add
                            alpha[t+y] = max(alpha[t+y],w_temp)
                        if q - w_i <= w_temp <= q-1:
                            b = b + c_add
        ts = ts + y_0
    return b

In [5]:
EC_BPI_raw = np.array([BPI_raw(electoral_college,i) for i in range(1,52)])
EC_BPI_raw

array([518714817081760, 357516461035922, 262324582416368, 243926123180044,
       163430671593954, 163430671593954, 145923747401144, 137210288981154,
       137210288981154, 128521091703182, 119854418693448, 111208580205692,
       102581928561098,  93972853943984,  93972853943984,  93972853943984,
        93972853943984,  85379780467994,  85379780467994,  85379780467994,
        85379780467994,  85379780467994,  76801162328032,  76801162328032,
        68235480794864,  68235480794864,  68235480794864,  59681240815152,
        59681240815152,  51136968067196,  51136968067196,  51136968067196,
        51136968067196,  51136968067196,  51136968067196,  42601206119710,
        42601206119710,  34072513921886,  34072513921886,  34072513921886,
        34072513921886,  34072513921886,  34072513921886,  34072513921886,
        25549462553022,  25549462553022,  25549462553022,  25549462553022,
        25549462553022,  25549462553022,  25549462553022])

In [6]:
EC_BPI_normalized = EC_BPI_raw / np.sum(EC_BPI_raw)
EC_BPI_normalized

array([0.11079641, 0.07636478, 0.05603199, 0.05210212, 0.03490845,
       0.03490845, 0.03116901, 0.02930783, 0.02930783, 0.02745184,
       0.02560066, 0.02375392, 0.02191129, 0.02007241, 0.02007241,
       0.02007241, 0.02007241, 0.01823694, 0.01823694, 0.01823694,
       0.01823694, 0.01823694, 0.01640457, 0.01640457, 0.01457496,
       0.01457496, 0.01457496, 0.01274779, 0.01274779, 0.01092275,
       0.01092275, 0.01092275, 0.01092275, 0.01092275, 0.01092275,
       0.00909953, 0.00909953, 0.00727782, 0.00727782, 0.00727782,
       0.00727782, 0.00727782, 0.00727782, 0.00727782, 0.00545731,
       0.00545731, 0.00545731, 0.00545731, 0.00545731, 0.00545731,
       0.00545731])

In [7]:
EC_BPI = EC_BPI_raw / 2 ** 50
EC_BPI

array([0.46071131, 0.31753841, 0.23299103, 0.21664992, 0.1451556 ,
       0.1451556 , 0.12960632, 0.12186722, 0.12186722, 0.11414966,
       0.10645211, 0.09877306, 0.09111106, 0.08346466, 0.08346466,
       0.08346466, 0.08346466, 0.07583248, 0.07583248, 0.07583248,
       0.07583248, 0.07583248, 0.06821313, 0.06821313, 0.06060528,
       0.06060528, 0.06060528, 0.05300759, 0.05300759, 0.04541875,
       0.04541875, 0.04541875, 0.04541875, 0.04541875, 0.04541875,
       0.03783747, 0.03783747, 0.03026247, 0.03026247, 0.03026247,
       0.03026247, 0.03026247, 0.03026247, 0.03026247, 0.02269248,
       0.02269248, 0.02269248, 0.02269248, 0.02269248, 0.02269248,
       0.02269248])